# Double/Debiased Machine Learning for the Partially Linear Regression Model

This is a simple implementation of Debiased Machine Learning for the Partially Linear Regression Model, which provides an application of DML inference to determine the causal effect of countries' intitial wealth on the rate of economic growth.


Reference:

- https://arxiv.org/abs/1608.00060
- https://www.amazon.com/Business-Data-Science-Combining-Accelerate/dp/1260452778

The code is based on the book.

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import warnings
from sklearn.base import BaseEstimator
import statsmodels.formula.api as smf
warnings.simplefilter('ignore')

np.random.seed(123)

In [2]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/GrowthData.csv"
data = pd.read_csv(file)
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]  # get rid of index column
data.shape

(90, 63)

In [3]:
data.head()

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283


Construct the outcome variable, the treatment variable and the matrix $x$ that includes the control variables.

In [4]:
y = data["Outcome"]
d = data["gdpsh465"]
x = data[data.columns[~data.columns.isin(['gdpsh465', 'intercept', 'Outcome'])]]

In [5]:
# some summary statistics
print("The length of y is: ", y.shape[0])
print("The number of features in x is: ", x.shape[1])
# naive OLS
all_columns = "+".join(data.iloc[:, 2:].columns)
my_formula = "Outcome ~ " + all_columns
ols_naive = smf.ols(formula=my_formula, data=data).fit()
print("Naive OLS that uses all features w/o cross-fitting Y ~ D+X yields: ",
      ols_naive.params[1], "(", ols_naive.bse[1], ")")

The length of y is:  90
The number of features in x is:  60
Naive OLS that uses all features w/o cross-fitting Y ~ D+X yields:  -0.009377988783847652 ( 0.02988772636631458 )


## DML algorithm

Here we perform estimation and inference of predictive coefficient $\alpha$ in the partially linear statistical model,
$$
Y = D\alpha + g(X) + U, \quad E (U | D, X) = 0.
$$
For $\tilde Y = Y- E(Y|X)$ and $\tilde D= D- E(D|X)$, we can write
$$
\tilde Y = \alpha \tilde D + U, \quad E (U |\tilde D) =0.
$$
Parameter $\alpha$ is then estimated using cross-fitting approach to obtain the residuals $\tilde D$ and $\tilde Y$.
The algorithm comsumes $Y, D, X$, and machine learning methods for learning the residuals $\tilde Y$ and $\tilde D$, where
the residuals are obtained by cross-validation (cross-fitting).

The statistical parameter $\alpha$ has a causal interpretation of being the effect of $D$ on $Y$ in the causal DAG $$ D\to Y, \quad X\to (D,Y)$$ or the counterfactual outcome model with conditionally exogenous (conditionally random) assignment of treatment $D$ given $X$:
$$
Y(d) = d\alpha + g(X) + U(d),\quad  U(d) \text{ indep } D |X, \quad Y = Y(D), \quad U = U(D).
$$


In [6]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False, time=None, clu=None, cluster=True):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    time: array of time indices, eg [0,1,...,T-1,0,1,...,T-1,...,0,1,...,T-1]
    clu: array of cluster indices, eg [1073, 1073, 1073, ..., 5055, 5055, 5055, 5055]
    cluster: bool, whether to use clustered standard errors

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    if cluster:
        # final stage ols clustered
        dml_data = pd.concat([clu, pd.Series(time),
                              pd.Series(resy, name='resy'),
                              pd.Series(resD, name='resD')], axis=1)
    else:
        # final stage ols nonclustered
        dml_data = pd.concat([pd.Series(resy, name='resy'),
                              pd.Series(resD, name='resD')], axis=1)

    if cluster:
        # clustered standard errors
        ols_mod = smf.ols(formula='resy ~ 1 + resD', data=dml_data)
        ols_mod = ols_mod.fit(cov_type='cluster', cov_kwds={"groups": dml_data['CountyCode']})
    else:
        # regular ols
        ols_mod = smf.ols(formula='resy ~ 1 + resD', data=dml_data).fit()

    point = ols_mod.params[1]
    stderr = ols_mod.bse[1]
    epsilon = ols_mod.resid

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [7]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2))  # RMSE of model that predicts treatment D
                         }, index=[name])

We now run through DML using as first stage models:
 1. OLS
 2. (Rigorous) Lasso
 3. Random Forests
 4. Mix of Random Forest and Lasso

Run the following command to install hdmpy for rigorous lasso:

In [8]:
!pip install multiprocess


!git clone https://github.com/maxhuppertz/hdmpy.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.7 MB/s eta 0:00:00
Cloning into 'hdmpy'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 70 (delta 39), reused 52 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (70/70), 25.30 KiB | 4.22 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [9]:
import hdmpy


class RLasso(BaseEstimator):

    def __init__(self, *, post=True):
        self.post = post

    def fit(self, X, y):
        self.rlasso_ = hdmpy.rlasso(X, y, post=self.post)
        return self

    def predict(self, X):
        return np.array(X) @ np.array(self.rlasso_.est['beta']).flatten() + np.array(self.rlasso_.est['intercept'])


def lasso_model():
    return RLasso(post=False)

In [10]:
# DML with OLS:
modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())
result_OLS = dml(x, d, y, modely, modeld, nfolds=10, classifier=False, cluster=False)
table_OLS = summary(*result_OLS, x, d, y, name='OLS')

# DML with RLasso:
modely = make_pipeline(StandardScaler(), RLasso(post=False))
modeld = make_pipeline(StandardScaler(), RLasso(post=False))
result_RLasso = dml(x, d, y, modely, modeld, nfolds=10, classifier=False, cluster=False)
table_RLasso = summary(*result_RLasso, x, d, y, name='Lasso')


# DML with Random Forests
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
result_RF = dml(x, d, y, modely, modeld, nfolds=10, classifier=False, cluster=False)
table_RF = summary(*result_RF, x, d, y, name='RF')

# DML with Mix:
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RLasso(post=False))
result_mix = dml(x, d, y, modely, modeld, nfolds=10, classifier=False, cluster=False)
table_mix = summary(*result_mix, x, d, y, name='RF/Lasso Mix')

In [11]:
table = pd.concat([table_OLS, table_RLasso, table_RF, table_mix], axis=0)

In [12]:
print(table)

              estimate    stderr    rmse y    rmse D
OLS           0.013017  0.016336  0.071830  0.467264
Lasso        -0.041312  0.014700  0.052376  0.363859
RF           -0.044985  0.012322  0.046042  0.372082
RF/Lasso Mix -0.044739  0.012619  0.046042  0.363859
